In [9]:
from lattice_surgery import Patch, LatticeSurgerySchedule

In [10]:
device = [[(r, c) for c in range(3)] for r in range(8)]

In [11]:
schedule = LatticeSurgerySchedule()

In [12]:
schedule.inject_T((0, 0))
schedule.inject_T((0, 1))
schedule.inject_T((0, 2))
schedule.inject_T((0, 3))
schedule.inject_T((2, 0))
schedule.inject_T((2, 1))
schedule.inject_T((2, 2))
schedule.inject_T((2, 4))
schedule.add_op([(0,0), (0,1), (0,2), (0,3), (2,0), (2,1), (2,2), (2,4)], [(1,0), (1,1), (1,2), (1,3), (1,4)], advance_time=True)

schedule.inject_T((0, 4))
schedule.inject_T((0, 5))
schedule.inject_T((2, 3))
schedule.inject_T((2, 5))
schedule.add_op([(0,0), (0,1), (0,4), (0,5), (2,0), (2,1), (2,3), (2,5)], [(1,0), (1,1), (1,2), (1,3), (1,4), (1,5)], advance_time=True)

schedule.inject_T((0, 6))
schedule.inject_T((2, 6))
schedule.add_op([(0,0), (0,2), (0,4), (0,6), (2,0), (2,2), (2,3), (2,6)], [(1,0), (1,1), (1,2), (1,3), (1,4), (1,5), (1,6)], advance_time=True)

schedule.inject_T((2, 7))
schedule.add_op([(0,0), (0,3), (0,5), (0,6), (2,1), (2,2), (2,3), (2,7)], [(1,0), (1,1), (1,2), (1,3), (1,4), (1,5), (1,6), (1,7)], advance_time=True)

schedule.inject_T((0, 7))
schedule.add_op([(0,0), (0,1), (0,2), (0,3), (0,4), (0,5), (0,6), (0,7)], [(1,0), (1,1), (1,2), (1,3), (1,4), (1,5), (1,6), (1,7)])

In [13]:
schedule.layers

[[('INJECT_T', (0, 0)),
  ('INJECT_T', (0, 1)),
  ('INJECT_T', (0, 2)),
  ('INJECT_T', (0, 3)),
  ('INJECT_T', (2, 0)),
  ('INJECT_T', (2, 1)),
  ('INJECT_T', (2, 2)),
  ('INJECT_T', (2, 4)),
  ('MERGE',
   [(0, 0),
    (0, 1),
    (0, 2),
    (0, 3),
    (2, 0),
    (2, 1),
    (2, 2),
    (2, 4),
    (1, 0),
    (1, 1),
    (1, 2),
    (1, 3),
    (1, 4)])],
 [('INJECT_T', (0, 4)),
  ('INJECT_T', (0, 5)),
  ('INJECT_T', (2, 3)),
  ('INJECT_T', (2, 5)),
  ('MERGE',
   [(0, 0),
    (0, 1),
    (0, 4),
    (0, 5),
    (2, 0),
    (2, 1),
    (2, 3),
    (2, 5),
    (1, 0),
    (1, 1),
    (1, 2),
    (1, 3),
    (1, 4),
    (1, 5)])],
 []]